In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('tree').getOrCreate()

In [3]:
data = spark.read.csv('College.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
data.head(1)[0]

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

## Format Data

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler = VectorAssembler(inputCols=['Apps',
                                     'Accept',
                                     'Enroll',
                                     'Top10perc',
                                     'Top25perc',
                                     'F_Undergrad',
                                     'P_Undergrad',
                                     'Outstate',
                                     'Room_Board',
                                     'Books',
                                     'Personal',
                                     'PhD',
                                     'Terminal',
                                     'S_F_Ratio',
                                     'perc_alumni',
                                     'Expend',
                                     'Grad_Rate'],
                             outputCol='features')

In [11]:
output = assembler.transform(data)

## Change the Private column instead of Yes/No, to 1 or 0

In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [14]:
output_fixed = indexer.fit(output).transform(output)

In [15]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [16]:
final_data = output_fixed.select('features','PrivateIndex')

In [17]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [19]:
from pyspark.ml import Pipeline

In [36]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(numTrees=150, labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [38]:
#dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
#gbt_model = gbt.fit(train_data)

In [39]:
#dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
#gbt_preds = gbt_model.transform(test_data)

In [44]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [45]:
my_bin_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [46]:
# Decision Tree Classifier
print('DTC')
print(my_bin_eval.evaluate(dtc_preds))
#Area under the curve

DTC
0.8568571428571429


In [47]:
# Random Forest Classifier
print('RFC')
print(my_bin_eval.evaluate(rfc_preds))

RFC
0.9859428571428572


In [30]:
# Gradient Boosting Classifier
print('GBT')
print(my_bin_eval.evaluate(gbt_preds))

GBT
0.9573714285714282


In [31]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [32]:
rfc_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [42]:
# For previous versions of Spark, we should create a new evaluation so we can see the gbt predictions
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')

In [43]:
# Gradient Boosting Classifier
print('GBT')
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.8814285714285713


In [48]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [51]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [52]:
rfc_acc

0.9466666666666667